In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas_datareader as pdr
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# 맥 폰트 깨짐 이슈 해결
from matplotlib import rc
%matplotlib inline
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [5]:
train = pd.read_csv('titanic/train.csv')
test = pd.read_csv('titanic/test.csv')

In [6]:
# 각 그룹별 통계치 => 결측값 대체
data = {
    'Gender' : ['Male', 'Male', 'Female', 'Female', 'Male','Male','Male','Male','Female', 'Female'],
    'Fruit' : ['Apple',np.nan,'Banana','Apple',np.nan,'Strawberry','Banana','Banana','Apple', np.nan],
    'Age' : [20, 14, 13, 22, 14, 32, 31, 22,23, 33],
    'Height' : [187,np.nan,155,165,177,np.nan,170,179,np.nan, 166]
}
df = pd.DataFrame(data)

In [7]:
df

,Gender,Fruit,Age,Height
0,Male,Apple,20,187.0
1,Male,NaN,14,NaN
2,Female,Banana,13,155.0
3,Female,Apple,22,165.0
4,Male,NaN,14,177.0
5,Male,Strawberry,32,NaN
6,Male,Banana,31,170.0
7,Male,Banana,22,179.0
8,Female,Apple,23,NaN
9,Female,NaN,33,166.0


In [12]:
df[df['Gender']=='Male']['Fruit'].mode()

0    Banana
Name: Fruit, dtype: object

In [13]:
df[df['Gender']=='Female']['Fruit'].mode()

0    Apple
Name: Fruit, dtype: object

In [14]:
df[df['Gender']=='Male']

,Gender,Fruit,Age,Height
0,Male,Apple,20,187.0
1,Male,NaN,14,NaN
4,Male,NaN,14,177.0
5,Male,Strawberry,32,NaN
6,Male,Banana,31,170.0
7,Male,Banana,22,179.0


In [16]:
df[df['Gender']=='Male']['Height'].mean() #178.25

178.25

In [17]:
df[df['Gender']=='Female']['Height'].mean() # 162.0

162.0

In [ ]:
# 1) 범주형 결측값 대체 방법 : 성별로 그룹화 -> 빈도수가 가장 높은 값으로 대체
# 2) 연속형 결측값 대체 방법 : 성별로 그룹화 -> 통계값(평균, 중위수,...)으로 대체

In [18]:
df

,Gender,Fruit,Age,Height
0,Male,Apple,20,187.0
1,Male,NaN,14,NaN
2,Female,Banana,13,155.0
3,Female,Apple,22,165.0
4,Male,NaN,14,177.0
5,Male,Strawberry,32,NaN
6,Male,Banana,31,170.0
7,Male,Banana,22,179.0
8,Female,Apple,23,NaN
9,Female,NaN,33,166.0


In [19]:
# transform 함수 : 그룹별로 계산된 통계값으로 대체
# Fruit열의 결측값 대체
# - Gender가 Male인 경우에는 Male그룹에 대한 Fruit열 값의 최빈값으로 대체
# - Gender가 Female인 경우에는 Female그룹에 대한 Fruit열 값의 최빈값으로 대체

In [25]:
# df의 Fruit열에 대한 결측값을 그룹별 통계값으로 대체
# df['Fruit'].fillna(대체값,inplace=True)

df['Fruit'].fillna(df.groupby('Gender')['Fruit'].transform(lambda x:x.value_counts().idxmax()),inplace=True)
# x에는 Gender별 Fruit열이 전달됨
df

,Gender,Fruit,Age,Height
0,Male,Apple,20,187.0
1,Male,Banana,14,NaN
2,Female,Banana,13,155.0
3,Female,Apple,22,165.0
4,Male,Banana,14,177.0
5,Male,Strawberry,32,NaN
6,Male,Banana,31,170.0
7,Male,Banana,22,179.0
8,Female,Apple,23,NaN
9,Female,Apple,33,166.0


In [22]:
df['Gender'].value_counts()

Gender
Male      6
Female    4
Name: count, dtype: int64

In [23]:
df['Gender'].value_counts().idxmax()

'Male'

In [28]:
# df의 Height열 대해 결측값을 그룹별로 평균으로 대체 using transform func
# 방법1
df['Height'].fillna(df.groupby('Gender')['Height'].transform(lambda x:x.mean()),inplace=True)

# 방법2
df.Height.fillna(df.groupby('Gender')['Height'].transform('mean'), inplace=True)

/var/folders/bs/rn4q13z924s9_wzlq41h6grc0000gn/T/ipykernel_1447/3696499503.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Height'].fillna(df.groupby('Gender')['Height'].transform(lambda x:x.mean()),inplace=True)
/var/folders/bs/rn4q13z924s9_wzlq41h6grc0000gn/T/ipykernel_1447/3696499503.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate

서포터 백터 머신(SVM)
- 데이터 분류(예측)를 위하여 마진(margin)이 최대가 되는 결정 경계선을 찾아내는 머신러닝 방법
- optimal(*) 함수뒤에 *이 붙음 / 최적의 결정 경계선 찾기

단어
- 결정 경계선
- 서포트 백터
- 마진
- 비용(cost=c / gamma)
- 커널 트릭 : 선형 분리 가능하게 함
- hyper parameter : 사용자가 설정하는 값

결정 경계선
- 서로 다른 분류값을 결정하는 경계
- SVM 학습의 최종목적은 주어진 데이터를 통해 최적의 결정 경계를 찾는 것
- 데이터의 벡터 공간을 N차원이라고 할 경우, 결정 경계 식
- 결정 경계 = N -1 차원
- 2차원에서는 선으로 구분 지어서 1차원 / 3차원에서는 면으로 구분지어서 2차원이다

커널 트릭
- 커널을 사용하지 않고 데이터를분류

가오시안 RBF 커널
- 데이터포인트에 적용되는 가우시안 함수의 표준편차를 조정함으로써 결정 결계 곡률을 조정
- 표준 조정 변수를 감마라고 부름

파라미터 튜닝
- 정확도를 향상시키기 위해 조절 가능한 파라미터는 비용과 감마



In [29]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [31]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
